## 初识pandas

首先，pandas重点提供了两种数据结构：
- Series

  序列，一维数据，是对NumPy的一维数组的封装，但是相较于NumPy使用整型下标，它使用自定义(比如有意义的字符串)的索引(index)
- DataFrame

  数据框，二维数据，是对NumPy的二维数组的封装，但是相较于NumPy使用整型下标，它可以使用自定义的索引(index)和列名(column)

在使用index,column之外，这两个封装额外还附带了更多趁手的方法，比如：
- describe ——— 快速计算数据的各种描述性统计值(均值、总和、中位数、四分位数等等)
- unique ——— 数据的独立值列表(比如想知道某个特征的所有取值可能)
- value_count ——— 各个值的计数
- hist ——— 直接绘制直方图
- plot ———— 对matplotlib进行了简单的封装，可以快速地进行简单的数据绘图

其次，pandas还提供了很多非常有用的处理数据时的小工具，比如:
- 便捷的Ｉ/O ——— 提供了直接读取Excel、CSV等常见的数据文件工具
- SQL的功能 ——— 提供了groupby，join等功能
- Excel的功能 ——— 透视表(pivot table)功能
- 日期相关功能 ——— 直观到像自然语言，不必费劲地去理解Python自带的日期库

接下来，我们大致领略一下pandas的核心功能
首先导入需要用的模块:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 创建对象(Object Creation)
通过传入一个列表数据，pandas可以创建一个使用默认整型作为索引的Series对象

In [2]:
s = pd.Series([1,2,3,4,np.nan,6,7,8])
s

0    1.0
1    2.0
2    3.0
3    4.0
4    NaN
5    6.0
6    7.0
7    8.0
dtype: float64

可以构建一个使用日期和标签作为索引的DataFrame对象

In [3]:
dates = pd.date_range('20190819', periods = 6)
dates

DatetimeIndex(['2019-08-19', '2019-08-20', '2019-08-21', '2019-08-22',
               '2019-08-23', '2019-08-24'],
              dtype='datetime64[ns]', freq='D')

In [4]:
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=tuple('ABCD')) # tuple('ABCD') is short for ('A','B','C','D')
df

,A,B,C,D
2019-08-19,-0.015907,0.126182,-0.361353,0.017073
2019-08-20,-1.905604,0.653322,0.019763,0.887715
2019-08-21,-0.704727,-2.072731,0.192907,0.513866
2019-08-22,0.912454,1.753704,0.308529,-0.079383
2019-08-23,-2.443910,-1.407360,-0.680321,0.487384
2019-08-24,-0.043260,-0.410198,0.596697,1.739272


也可以使用一个字典(dict)来创建一个DataFrame对象，而且它会自动应用NumPy的广播

In [5]:
df2 = pd.DataFrame({'A' : 1.,
                    'B' : pd.Timestamp('20171026'),
                    'C' : pd.Series(1, index=list(range(4)),dtype='float32'),
                    'D' : np.array([3] * 4, dtype = 'int32'),
                    'E' : pd.Categorical(["test", "train", "test", "train"]),
                    'F' : 'foo'})
df2

,A,B,C,D,E,F
0,1.0,2017-10-26,1.0,3,test,foo
1,1.0,2017-10-26,1.0,3,train,foo
2,1.0,2017-10-26,1.0,3,test,foo
3,1.0,2017-10-26,1.0,3,train,foo


In [6]:
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
dtype: object

In [7]:
df2.C # 直接使用标签来选择列，等价于df2['C']

0    1.0
1    1.0
2    1.0
3    1.0
Name: C, dtype: float32

## 查看数据(viewing data)
比如想看看一个DataFrame的头部和尾部：

In [8]:
df.head() #head默认参数为5，前5行

,A,B,C,D
2019-08-19,-0.015907,0.126182,-0.361353,0.017073
2019-08-20,-1.905604,0.653322,0.019763,0.887715
2019-08-21,-0.704727,-2.072731,0.192907,0.513866
2019-08-22,0.912454,1.753704,0.308529,-0.079383
2019-08-23,-2.443910,-1.407360,-0.680321,0.487384


In [9]:
df.tail(3)

,A,B,C,D
2019-08-22,0.912454,1.753704,0.308529,-0.079383
2019-08-23,-2.443910,-1.407360,-0.680321,0.487384
2019-08-24,-0.043260,-0.410198,0.596697,1.739272


我们也可以看看索引、列名、以及底层的numpy数据都是什么样

In [10]:
df.index

DatetimeIndex(['2019-08-19', '2019-08-20', '2019-08-21', '2019-08-22',
               '2019-08-23', '2019-08-24'],
              dtype='datetime64[ns]', freq='D')

In [11]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [12]:
df.values

array([[-0.01590739,  0.12618176, -0.36135317,  0.01707306],
       [-1.9056039 ,  0.65332242,  0.01976319,  0.88771453],
       [-0.70472688, -2.07273149,  0.19290655,  0.5138663 ],
       [ 0.91245442,  1.75370427,  0.30852851, -0.07938254],
       [-2.44390964, -1.40736047, -0.6803206 ,  0.48738394],
       [-0.04325951, -0.41019775,  0.59669687,  1.73927197]])

而且我们可以通过describe()方法来快速地看看数据的概括统计：
- count:统计数量
- mean: 均值
- std: 标准差
- min: 最小值
- 25%: 四分位数
- 50%: 中位数
- 75%: 四分之三位数
- max: 最大值

In [13]:
df.describe()

,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,-0.700159,-0.226180,0.012704,0.594321
std,1.264439,1.389707,0.465034,0.663625
min,-2.443910,-2.072731,-0.680321,-0.079383
25%,-1.605385,-1.158070,-0.266074,0.134651
50%,-0.373993,-0.142008,0.106335,0.500625
75%,-0.022745,0.521537,0.279623,0.794252
max,0.912454,1.753704,0.596697,1.739272


In [14]:
df.T # 转置

,2019-08-19,2019-08-20,2019-08-21,2019-08-22,2019-08-23,2019-08-24
A,-0.015907,-1.905604,-0.704727,0.912454,-2.443910,-0.043260
B,0.126182,0.653322,-2.072731,1.753704,-1.407360,-0.410198
C,-0.361353,0.019763,0.192907,0.308529,-0.680321,0.596697
D,0.017073,0.887715,0.513866,-0.079383,0.487384,1.739272


以某一个轴排序，注意这是按照轴自己的值来排序，比如按照列名来排序：

In [15]:
df.sort_index(axis = 1, ascending = False)

,D,C,B,A
2019-08-19,0.017073,-0.361353,0.126182,-0.015907
2019-08-20,0.887715,0.019763,0.653322,-1.905604
2019-08-21,0.513866,0.192907,-2.072731,-0.704727
2019-08-22,-0.079383,0.308529,1.753704,0.912454
2019-08-23,0.487384,-0.680321,-1.407360,-2.443910
2019-08-24,1.739272,0.596697,-0.410198,-0.043260


In [16]:
df.sort_index(axis = 0, ascending = True)

,A,B,C,D
2019-08-19,-0.015907,0.126182,-0.361353,0.017073
2019-08-20,-1.905604,0.653322,0.019763,0.887715
2019-08-21,-0.704727,-2.072731,0.192907,0.513866
2019-08-22,0.912454,1.753704,0.308529,-0.079383
2019-08-23,-2.443910,-1.407360,-0.680321,0.487384
2019-08-24,-0.043260,-0.410198,0.596697,1.739272


也可以按照数据的值来排序：

In [17]:
df.sort_values(by = 'B')

,A,B,C,D
2019-08-21,-0.704727,-2.072731,0.192907,0.513866
2019-08-23,-2.443910,-1.407360,-0.680321,0.487384
2019-08-24,-0.043260,-0.410198,0.596697,1.739272
2019-08-19,-0.015907,0.126182,-0.361353,0.017073
2019-08-20,-1.905604,0.653322,0.019763,0.887715
2019-08-22,0.912454,1.753704,0.308529,-0.079383


## 选择数据(Selection)
pandas数据访问方法，如：.at, .iat, .loc, .iloc以及.ix

https://www.cnblogs.com/anliux/p/10805103.html

这里给出索引(index/selection)方法的概览：

操作             |语法                       | 结果类型
---------------:|--------------------------:|--------------:
选择列           |df[col]                    | Series
选择行           |df.loc[label]              | Series
选择列、行        |df.loc[index,column]      |  DataFrame
使用位置选择行    |df.iloc[loc]               |  Series
使用位置选择行、列 |df.iloc[v_loc, h_loc]      |  DataFrame
行切片           |df[5:10]/df[index1:index2] |  DataFrame
使用布尔向量选择行 |df[bool_vec]               |  DataFrame

## 访问数据(Getting)
选择某一列，会返回一个Series对象，等价与df.A

In [18]:
df['A'] #df.A

2019-08-19   -0.015907
2019-08-20   -1.905604
2019-08-21   -0.704727
2019-08-22    0.912454
2019-08-23   -2.443910
2019-08-24   -0.043260
Freq: D, Name: A, dtype: float64

使用切片的方式获得某些行：

In [19]:
df[0:3]

,A,B,C,D
2019-08-19,-0.015907,0.126182,-0.361353,0.017073
2019-08-20,-1.905604,0.653322,0.019763,0.887715
2019-08-21,-0.704727,-2.072731,0.192907,0.513866


这种方法和NumPy的二维数组没有什么区别，但是在pandas中我们可以直接使用索引的值，更加自然：

In [20]:
df['20190819':'20190821'] # 注意这种索引值，结束值也会被返回，因为它们不是整型数字

,A,B,C,D
2019-08-19,-0.015907,0.126182,-0.361353,0.017073
2019-08-20,-1.905604,0.653322,0.019763,0.887715
2019-08-21,-0.704727,-2.072731,0.192907,0.513866


## 使用标签选择数据
loc方法使用索引或者列标签来选择数据：

In [21]:
df.loc[dates[0]]

A   -0.015907
B    0.126182
C   -0.361353
D    0.017073
Name: 2019-08-19 00:00:00, dtype: float64

In [22]:
df.loc[:, ['A', 'B']] # 指定要A,B两列

,A,B
2019-08-19,-0.015907,0.126182
2019-08-20,-1.905604,0.653322
2019-08-21,-0.704727,-2.072731
2019-08-22,0.912454,1.753704
2019-08-23,-2.443910,-1.407360
2019-08-24,-0.043260,-0.410198


In [23]:
df.loc['20190819':'20190824', ['A', 'B']] # index是可以被切片的

,A,B
2019-08-19,-0.015907,0.126182
2019-08-20,-1.905604,0.653322
2019-08-21,-0.704727,-2.072731
2019-08-22,0.912454,1.753704
2019-08-23,-2.443910,-1.407360
2019-08-24,-0.043260,-0.410198


In [24]:
df.loc['20190819':'20190824', 'A' : 'B'] # columns也可以被切片

,A,B
2019-08-19,-0.015907,0.126182
2019-08-20,-1.905604,0.653322
2019-08-21,-0.704727,-2.072731
2019-08-22,0.912454,1.753704
2019-08-23,-2.443910,-1.407360
2019-08-24,-0.043260,-0.410198


In [25]:
df.loc[dates[0], 'A'] # 获取特定位置的数据

-0.015907387694743178

In [26]:
df.at[dates[0], 'A'] # 和上面一行等价，某一点的数据

-0.015907387694743178

## 使用位置来选择(selection by position)
iloc使用位置来选择数据，基本类似于NumPy的方法

In [27]:
df.iloc[3] #第三行数据

A    0.912454
B    1.753704
C    0.308529
D   -0.079383
Name: 2019-08-22 00:00:00, dtype: float64

In [28]:
df.iloc[3:5, 0:2] #第4、5行,第1、2列

,A,B
2019-08-22,0.912454,1.753704
2019-08-23,-2.443910,-1.407360


In [29]:
df.iloc[[1,2,4], [0,2]] #数组方式，第1、2、4行,第1、2列

,A,C
2019-08-20,-1.905604,0.019763
2019-08-21,-0.704727,0.192907
2019-08-23,-2.443910,-0.680321


In [32]:
df.iloc[:, 1:3]

,B,C
2019-08-19,0.126182,-0.361353
2019-08-20,0.653322,0.019763
2019-08-21,-2.072731,0.192907
2019-08-22,1.753704,0.308529
2019-08-23,-1.407360,-0.680321
2019-08-24,-0.410198,0.596697


iloc也可以表示某个位置的数据值

In [33]:
df.iloc[1,1] #等价于df.iat[1,1]

0.6533224162542729

## 布尔索引
这种方法与NumPy类似

In [35]:
df.A > 0

2019-08-19    False
2019-08-20    False
2019-08-21    False
2019-08-22     True
2019-08-23    False
2019-08-24    False
Freq: D, Name: A, dtype: bool

In [34]:
df[df.A > 0] #选择行

,A,B,C,D
2019-08-22,0.912454,1.753704,0.308529,-0.079383


In [36]:
df > 0

,A,B,C,D
2019-08-19,False,True,False,True
2019-08-20,False,True,True,True
2019-08-21,False,False,True,True
2019-08-22,True,True,True,False
2019-08-23,False,False,False,True
2019-08-24,False,False,True,True


In [37]:
df[df > 0]

,A,B,C,D
2019-08-19,NaN,0.126182,NaN,0.017073
2019-08-20,NaN,0.653322,0.019763,0.887715
2019-08-21,NaN,NaN,0.192907,0.513866
2019-08-22,0.912454,1.753704,0.308529,NaN
2019-08-23,NaN,NaN,NaN,0.487384
2019-08-24,NaN,NaN,0.596697,1.739272


我们可以使用isin()方法来进行过滤，对于非数值型数据很有用

In [38]:
df2 = df.copy()
df2['E'] = ['one', 'one', 'two', 'three', 'four', 'five']
df2

,A,B,C,D,E
2019-08-19,-0.015907,0.126182,-0.361353,0.017073,one
2019-08-20,-1.905604,0.653322,0.019763,0.887715,one
2019-08-21,-0.704727,-2.072731,0.192907,0.513866,two
2019-08-22,0.912454,1.753704,0.308529,-0.079383,three
2019-08-23,-2.443910,-1.407360,-0.680321,0.487384,four
2019-08-24,-0.043260,-0.410198,0.596697,1.739272,five


In [40]:
df2['E'].isin(['two', 'four'])

2019-08-19    False
2019-08-20    False
2019-08-21     True
2019-08-22    False
2019-08-23     True
2019-08-24    False
Freq: D, Name: E, dtype: bool

In [42]:
df2[df2['E'].isin(['two', 'four'])]

,A,B,C,D,E
2019-08-21,-0.704727,-2.072731,0.192907,0.513866,two
2019-08-23,-2.443910,-1.407360,-0.680321,0.487384,four


## 赋值(setting)

In [45]:
s1 = pd.Series([1,2,3,4,5,6], index = pd.date_range('20190819', periods = 6))
s1

2019-08-19    1
2019-08-20    2
2019-08-21    3
2019-08-22    4
2019-08-23    5
2019-08-24    6
Freq: D, dtype: int64

In [46]:
df['F'] = s1
df

,A,B,C,D,F
2019-08-19,-0.015907,0.126182,-0.361353,0.017073,1
2019-08-20,-1.905604,0.653322,0.019763,0.887715,2
2019-08-21,-0.704727,-2.072731,0.192907,0.513866,3
2019-08-22,0.912454,1.753704,0.308529,-0.079383,4
2019-08-23,-2.443910,-1.407360,-0.680321,0.487384,5
2019-08-24,-0.043260,-0.410198,0.596697,1.739272,6


In [47]:
df.at[dates[0], 'A'] = 0 #第0行，第1列设置成0
df

,A,B,C,D,F
2019-08-19,0.000000,0.126182,-0.361353,0.017073,1
2019-08-20,-1.905604,0.653322,0.019763,0.887715,2
2019-08-21,-0.704727,-2.072731,0.192907,0.513866,3
2019-08-22,0.912454,1.753704,0.308529,-0.079383,4
2019-08-23,-2.443910,-1.407360,-0.680321,0.487384,5
2019-08-24,-0.043260,-0.410198,0.596697,1.739272,6


In [48]:
df.iat[0, 1] = 0
df

,A,B,C,D,F
2019-08-19,0.000000,0.000000,-0.361353,0.017073,1
2019-08-20,-1.905604,0.653322,0.019763,0.887715,2
2019-08-21,-0.704727,-2.072731,0.192907,0.513866,3
2019-08-22,0.912454,1.753704,0.308529,-0.079383,4
2019-08-23,-2.443910,-1.407360,-0.680321,0.487384,5
2019-08-24,-0.043260,-0.410198,0.596697,1.739272,6


In [49]:
df.loc[:, 'D'] = np.array([5] * len(df))
df

,A,B,C,D,F
2019-08-19,0.000000,0.000000,-0.361353,5,1
2019-08-20,-1.905604,0.653322,0.019763,5,2
2019-08-21,-0.704727,-2.072731,0.192907,5,3
2019-08-22,0.912454,1.753704,0.308529,5,4
2019-08-23,-2.443910,-1.407360,-0.680321,5,5
2019-08-24,-0.043260,-0.410198,0.596697,5,6


## 缺失值
pandas一般使用np.nan表示缺失值，默认情况下不参与计算。
重建索引(reindexing)可以修改、增加、删除索引，而且会返回一份拷贝后的数据

In [50]:
df1 = df.reindex(index = dates[0:4], columns = list(df.columns) + ['E'])
df1.loc[dates[0]:dates[1], 'E'] = 1
df1

,A,B,C,D,F,E
2019-08-19,0.000000,0.000000,-0.361353,5,1,1.0
2019-08-20,-1.905604,0.653322,0.019763,5,2,1.0
2019-08-21,-0.704727,-2.072731,0.192907,5,3,NaN
2019-08-22,0.912454,1.753704,0.308529,5,4,NaN


删除含有缺失值的所有行

In [51]:
df1.dropna(how = 'any') #how = 'any', 某行有任一缺失值，该行都不要

,A,B,C,D,F,E
2019-08-19,0.000000,0.000000,-0.361353,5,1,1.0
2019-08-20,-1.905604,0.653322,0.019763,5,2,1.0


使用指定的值来替换缺失值

In [52]:
df1.fillna(value = 5)

,A,B,C,D,F,E
2019-08-19,0.000000,0.000000,-0.361353,5,1,1.0
2019-08-20,-1.905604,0.653322,0.019763,5,2,1.0
2019-08-21,-0.704727,-2.072731,0.192907,5,3,5.0
2019-08-22,0.912454,1.753704,0.308529,5,4,5.0


获取DataFrame中缺失值的掩码布尔矩阵

In [53]:
pd.isnull(df1) # 掩码布尔矩阵：哪些值是缺失的

,A,B,C,D,F,E
2019-08-19,False,False,False,False,False,False
2019-08-20,False,False,False,False,False,False
2019-08-21,False,False,False,False,False,True
2019-08-22,False,False,False,False,False,True


## 操作(Operations)
### 统计(stats)
操作一般都不计缺失值

In [55]:
df.mean() # 每一列的均值

A   -0.697508
B   -0.247211
C    0.012704
D    5.000000
F    3.500000
dtype: float64

In [57]:
df.mean(1) # 在另一个轴上，每一行的均值

2019-08-19    1.127729
2019-08-20    1.153496
2019-08-21    1.083090
2019-08-22    2.394937
2019-08-23    1.093682
2019-08-24    2.228648
Freq: D, dtype: float64

## 应用函数(apply)
对数据应用函数

In [58]:
df.apply(np.cumsum) # 每一列的累加和

,A,B,C,D,F
2019-08-19,0.000000,0.000000,-0.361353,5,1
2019-08-20,-1.905604,0.653322,-0.341590,10,3
2019-08-21,-2.610331,-1.419409,-0.148683,15,6
2019-08-22,-1.697876,0.334295,0.159845,20,10
2019-08-23,-4.141786,-1.073065,-0.520476,25,15
2019-08-24,-4.185046,-1.483263,0.076221,30,21


In [59]:
df.apply(lambda x: x.max() - x.min()) # 每一列的最大值-最小值

A    3.356364
B    3.826436
C    1.277017
D    0.000000
F    5.000000
dtype: float64

## 计数(histogramming)

In [60]:
s = pd.Series(np.random.randint(0, 7, size = 10))
s

0    4
1    3
2    5
3    0
4    6
5    3
6    2
7    6
8    4
9    2
dtype: int64

In [61]:
s.value_counts() #每个value出现的次数

6    2
4    2
3    2
2    2
5    1
0    1
dtype: int64

## 合并(merge) 
### 连接(concat)
pandas为了方便地将Series,DataFrame组合在一起，开发了各种各样的功能
使用concat()来将pandas的对象连接在一起

In [62]:
df = pd.DataFrame(np.random.randn(10,4))
df

,0,1,2,3
0,1.102174,-1.453703,2.656551,-0.414924
1,-0.193256,1.283319,0.669317,1.108068
2,-1.259005,-0.488508,-0.472873,-0.016043
3,-0.268243,0.125667,-2.570081,0.835589
4,-1.472655,1.452588,-0.453891,-0.406809
5,-0.275767,-1.341235,0.904750,0.044987
6,-0.307980,0.184910,0.837423,-1.124670
7,0.156126,2.075887,1.289221,2.272810
8,-0.019849,-1.161652,0.075107,0.859571
9,-0.944992,-0.882540,0.688862,-0.876590


In [63]:
pieces = [df[:3], df[3:7], df[7:]]
pieces

[          0         1         2         3
 0  1.102174 -1.453703  2.656551 -0.414924
 1 -0.193256  1.283319  0.669317  1.108068
 2 -1.259005 -0.488508 -0.472873 -0.016043,
           0         1         2         3
 3 -0.268243  0.125667 -2.570081  0.835589
 4 -1.472655  1.452588 -0.453891 -0.406809
 5 -0.275767 -1.341235  0.904750  0.044987
 6 -0.307980  0.184910  0.837423 -1.124670,
           0         1         2         3
 7  0.156126  2.075887  1.289221  2.272810
 8 -0.019849 -1.161652  0.075107  0.859571
 9 -0.944992 -0.882540  0.688862 -0.876590]

In [64]:
pd.concat(pieces)

,0,1,2,3
0,1.102174,-1.453703,2.656551,-0.414924
1,-0.193256,1.283319,0.669317,1.108068
2,-1.259005,-0.488508,-0.472873,-0.016043
3,-0.268243,0.125667,-2.570081,0.835589
4,-1.472655,1.452588,-0.453891,-0.406809
5,-0.275767,-1.341235,0.904750,0.044987
6,-0.307980,0.184910,0.837423,-1.124670
7,0.156126,2.075887,1.289221,2.272810
8,-0.019849,-1.161652,0.075107,0.859571
9,-0.944992,-0.882540,0.688862,-0.876590


## Join
SQL风格的操作

In [65]:
left = pd.DataFrame({'key':['foo', 'foo'], 'lval': [1, 2]})
right = pd.DataFrame({'key':['foo', 'foo'], 'rval': [4, 5]})
left

,key,lval
0,foo,1
1,foo,2


In [66]:
right

,key,rval
0,foo,4
1,foo,5


In [67]:
pd.merge(left, right, on = 'key')

,key,lval,rval
0,foo,1,4
1,foo,1,5
2,foo,2,4
3,foo,2,5


另一个可能更能演示的例子

In [68]:
left = pd.DataFrame({'key':['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key':['foo', 'bar'], 'rval': [4, 5]})
left

,key,lval
0,foo,1
1,bar,2


In [71]:
right

,key,rval
0,foo,4
1,bar,5


In [72]:
pd.merge(left, right, on = 'key')

,key,lval,rval
0,foo,1,4
1,bar,2,5


## 追加(append)
向DataFrame追加行

In [73]:
df = pd.DataFrame(np.random.randn(8,4), columns = ['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,-0.133957,-1.058909,0.854513,-0.394713
1,0.311837,0.666723,-0.211183,-0.187930
2,-0.311350,-0.274288,2.905623,0.575320
3,-0.991442,0.444133,0.302917,0.027540
4,0.677276,-0.921970,-0.749209,0.986745
5,-0.185768,-0.107232,0.190285,-1.718306
6,-0.498028,-2.041877,-0.210443,0.704943
7,-2.311726,0.275023,-0.392669,0.847770


In [75]:
s = df.iloc[3] #下标为3的行
s

A   -0.991442
B    0.444133
C    0.302917
D    0.027540
Name: 3, dtype: float64

In [76]:
df.append(s, ignore_index = True) #ignore_index表示忽略s的index

,A,B,C,D
0,-0.133957,-1.058909,0.854513,-0.394713
1,0.311837,0.666723,-0.211183,-0.187930
2,-0.311350,-0.274288,2.905623,0.575320
3,-0.991442,0.444133,0.302917,0.027540
4,0.677276,-0.921970,-0.749209,0.986745
5,-0.185768,-0.107232,0.190285,-1.718306
6,-0.498028,-2.041877,-0.210443,0.704943
7,-2.311726,0.275023,-0.392669,0.847770
8,-0.991442,0.444133,0.302917,0.027540


## 聚合(grouping)
"group by"指的是涉及如下一个或者多个步骤的一种处理过程：
- 根据某些条件将数据切分成一些组
- 对每个组独立地进行某种操作
- 将结果组合到一个数据结构中

In [78]:
df = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                         'foo', 'bar', 'foo', 'bar'],
                   'B': ['one', 'one', 'two', 'three',
                         'two', 'two', 'one', 'three'],
                   'C': np.random.randn(8),
                   'D': np.random.randn(8)})
df

,A,B,C,D
0,foo,one,-0.832725,-0.813722
1,bar,one,-0.687033,-0.641067
2,foo,two,-0.337180,-0.401284
3,bar,three,-0.612130,1.577137
4,foo,two,-2.049960,-1.054039
5,bar,two,-0.692122,-0.351802
6,foo,one,-1.697752,0.963477
7,bar,three,-0.057071,-1.290315


In [79]:
df.groupby('A').sum() #等价于SQL中的select A, sum(C), sum(D) from df group by A

,C,D
A,,
bar,-2.048357,-0.706048
foo,-4.917617,-1.305569


In [83]:
df.groupby(['A','B']).sum() #等价于SQL中的select A,B sum(C), sum(D) from df group by A,B

C         D
A   B                        
bar one   -0.687033 -0.641067
    three -0.669201  0.286822
    two   -0.692122 -0.351802
foo one   -2.530477  0.149755
    two   -2.387139 -1.455324

## 变更形状(reshaping)
### 堆叠(stack)

In [85]:
tuples = list(zip(['bar', 'bar', 'baz', 'baz',
                  'foo', 'foo', 'qux', 'qux'],
                 ['one', 'two', 'one', 'two',
                  'one', 'two', 'one', 'two']))
index = pd.MultiIndex.from_tuples(tuples, names = ['first', 'second']) #建立多维索引
df = pd.DataFrame(np.random.randn(8,2), index = index, columns = ['A', 'B'])
df2 = df[:4]
df2

A         B
first second                    
bar   one     0.379508  0.132329
      two    -2.819309  2.392931
baz   one    -0.478785 -0.483257
      two    -0.333391 -0.688026

stack()方法实际上是将DataFrame的某一级列转成行索引，它和我们在NumPy中学的stack不是一回事

In [86]:
stacked = df2.stack()
stacked

first  second   
bar    one     A    0.379508
               B    0.132329
       two     A   -2.819309
               B    2.392931
baz    one     A   -0.478785
               B   -0.483257
       two     A   -0.333391
               B   -0.688026
dtype: float64

如果一个DataFrame或者Series是堆叠的(它的索引是多重索引，即MultiIndex),那么可以使用unstack()来将索引的某一层转成列，默认转最后一级索引

In [87]:
stacked.unstack() # 默认转换最后一个索引

A         B
first second                    
bar   one     0.379508  0.132329
      two    -2.819309  2.392931
baz   one    -0.478785 -0.483257
      two    -0.333391 -0.688026

In [88]:
stacked.unstack(1) #指定转第二级(下标从0开始)

second        one       two
first                      
bar   A  0.379508 -2.819309
      B  0.132329  2.392931
baz   A -0.478785 -0.333391
      B -0.483257 -0.688026

In [90]:
stacked.unstack(0) #指定转第一级

first          bar       baz
second                      
one    A  0.379508 -0.478785
       B  0.132329 -0.483257
two    A -2.819309 -0.333391
       B  2.392931 -0.688026

## 数据透视表（Pivot Tables）
这个功能和Excel里面的数据透视表能够完成的功能几乎完全一样：选定维度来汇总数据，以从不同的角度来审视数据。

In [92]:
np.random.seed(1) #指定随机数种子，使得不同电脑上产生的随机数是相同的
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 3,
                   'B': ['A', 'B', 'C'] * 4,
                   'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 2, 
                   'D': np.random.randn(12),
                   'E': np.random.randn(12)})
df

,A,B,C,D,E
0,one,A,foo,1.624345,-0.322417
1,one,B,foo,-0.611756,-0.384054
2,two,C,foo,-0.528172,1.133769
3,three,A,bar,-1.072969,-1.099891
4,one,B,bar,0.865408,-0.172428
5,one,C,bar,-2.301539,-0.877858
6,two,A,foo,1.744812,0.042214
7,three,B,foo,-0.761207,0.582815
8,one,C,foo,0.319039,-1.100619
9,one,A,bar,-0.249370,1.144724


In [ ]:
#创建数据透视表
pd.pivot_table(df, values = 'D', index = ['A', 'B'], columns = ['C']) #对应于Excel,以A,B为行维度，以C为列维度，计算D列之和